# 2018-06-05 - Psychophysics using psychopy
On repasse sur un paradoxe plus classique de psychophysique.

On récupère les initiales :

In [4]:
from psychopy import visual, core, gui, event
import MotionClouds as MC
from psychopy import gui

exp_name = "Perception de l'orientation"
exp_info = {
            'Initiales ': ''
            }
subj_info = gui.DlgFromDict(dictionary=exp_info, title=exp_name)
print (exp_info)

{'Initiales ': 'HL'}


On défini la génération de MotionClouds :

In [1]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

#generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    imageio.imwrite('%s.png' % (theta) , im[:, :, 0])
    return im[:, :, 0]

On définit les paramètres et on teste :

In [2]:
from psychopy import visual, core, gui, event
import MotionClouds as MC
from psychopy import gui

test_length = 10 #number of trials
trial_length = 2.6 #length of trial, in second
fixation_length = 1.5 #length of fixation, in second
stim1_length = 0.5 #length of first stim, in second
stim2_length = 0.5 #length of the second stim, in second


#Psychopy window
win=visual.Window( fullscr = False, color = [0,0,100], winType = 'pygame', monitor = 'testMonitor')

#Answer list
ans_list = []

for trial in range(test_length) :
    #Shift to left (3) or to right (2) ?
    shift = random.randint(2,3)
    shift_dict = {'right' : 2,
                   'left' :  3}
    
    #MC generation
    B_thetas = np.pi*np.logspace(-7, 0, 100, base = 2)
    B_theta = B_thetas[random.randint(1,99)]
    
    MC1 = generate_random_cloud(np.pi/2, B_theta = B_theta)
    MC2 = generate_random_cloud(np.pi/3, B_theta = B_theta) #if shift = 2
    MC3 = generate_random_cloud(2 * (np.pi/3), B_theta = B_theta) #if shift = 3

    #MC to psychopy objects
    bitmap1 =visual.ImageStim(win, '%s.png' % str(np.pi/2), mask = 'circle')
    bitmap1.autolog = False
    bitmap2 =visual.ImageStim(win, '%s.png' % str(np.pi/3), mask = 'circle')
    bitmap2.autolog = False
    bitmap3 =visual.ImageStim(win, '%s.png' % str(2 * (np.pi/3)), mask = 'circle')
    bitmap3.autolog = False

    #Times the trial
    clock = core.Clock()
    while clock.getTime() < trial_length: 
        if 0.0 <= clock.getTime() < fixation_length : #fixation
            fixation = visual.ShapeStim(vertices = 'cross', win=win, size=0.2, pos=[0,0], fillColor = 'black')
            fixation.autolog = False
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length :
            bitmap1.draw()
        if fixation_length + stim1_length + 0.01 <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            if shift == 2 :
                bitmap2.draw()
            if shift == 3 :
                bitmap3.draw()
        win.flip()
    
    #display orientation choice
    msg = visual.TextStim(win, text = '<- LEFT or RIGHT ->')
    msg.draw()
    win.flip()
    
    #answer using the keyboard
    while True :
        ans = event.waitKeys()
        print(ans)
        if len(ans)>0 :
            break
    
    
    #Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, shift, shift_dict[ans[0]], B_theta])
    
win.close()

['right']
['left']
['right']
['right']
['right']
['left']
['right']
['left']
['left']
['right']


In [8]:
import pickle

pickle.dump(ans_list, open('Psychophys_discrim_%s.p' % exp_info['Initiales '], 'wb'))